# JSON_EXISTS with LAX and STRICT Error Handling
Updated: 2019-10-03

### Document Integrity
At a minimum, document correctness means that a document conforms to the JSON rules that were outlined in a previous notebook. For document validity, the document must contain certain key-value pairs and perhaps include some rules on how the key-value pairs are related.

From an ISO JSON perspective, there is no function which allows you to check the contents of a document against certain rules (like a schema-validation function). The validity of a document is left up to the developer.

However, there are mechanisms available that allow you to check the correctness of a document (does it meet JSON format requirements) and to determine whether or the document is valid from a business perspective (specific key-value pairs exists in the document).

In [ ]:
%run ../db2.ipynb
%run ../connection.ipynb

### Checking for Document Correctness
One of the advantages of using the new Db2 JSON functions is that you can store the data as either character (JSON) strings, or as binary (BSON) data. However, if you insert a document as a JSON character string, no checking will be done against the validity of the document until you attempt to use a JSON function against it. The following example attempts to retrieve the name field from a JSON document:

In [ ]:
%sql VALUES JSON_VALUE('{"name": George}','$.name')

From a JSON format perspective, this function will fail as the value George will be evaluated by JSON as a numeric since it is not quoted. All of the JSON functions will check the correctness of a document before evaluating any expressions. From this perspective, Db2 will enforce the syntactical correctness of any JSON document. If you need to check the contents of a document for particular key fields, then you must use the `JSON_EXISTS` function.

### Search Paths
When working with the Db2 JSON functions, you have the ability to provide a search path indicating how to find the JSON keys of interest in the document. What should happen in the event that your path expression cannot find the key that was requested? Should an error be raised, or a default value be returned? These are decisions which you should consider before you begin to access JSON documents in order to ensure that the results you get back follow a consistent standard.

To give this discussion some context, let's first define the type of "awkward" situations which an innocent path expression might potentially encounter when exploring a JSON document:

1. The document being searched is not a proper JSON document (i.e. it does not follow the standard JSON format)
2. The Path expression is invalid (i.e. it does not follow the standard path syntax)
3. There is a structural error in the document
   1. The key that you are attempting to retrieve does not exist
   2. The array value requested does not exist (out of bounds)

To allow you to have control over the behavior of your query, the standard includes two keywords, `lax` and `strict`, for the path expression which determine how structural errors will be handled. The ISO JSON SQL standard also provides mechanisms for dealing with missing keys, improper documents, or invalid path expressions (the `ON EMPTY` and `ON ERROR` clauses). 

### Lax Versus Strict Path Expressions
The beginning of every JSON path expression can contain one of two search modifiers: `lax` and `strict`. The search behavior can be explicitly modified using the lax or strict keyword before the JSON path:
```json
      strict $.stores[2].phone[1]
```
The default mode is `lax` for all Db2 JSON functions in Db2 except for `JSON_TABLE`.
The `lax` behavior is the tolerant one which will ignore structural differences between the path provided and the actual JSON document layout. A structural "difference" refers to a number of possible areas where the path might vary from the actual JSON document:

* The path specifies keys or levels that do not exist in the JSON document
* A missing object or element 
* Accessing an array without specifying the index value

When these types of errors occur, the output of the function under the default lax modifier will be to return a `NULL` value rather than an error. You can change this behavior by using the strict keyword or by using the `ON EMPTY` clause (discussed later).

The flow chart below summarizes Db2’s behavior for the `lax` and `strict` when an object is not found, and which clause is relevant; the default behavior for a missing value or error can be modified with additional `ON EMPTY` and `ON ERROR` clauses which will be covered in more detail below. 

The following document illustrates all of these structural issues.
```json
{
  "authors": 
    [
      {"first_name": "Paul",  "last_name" : "Bird"},
      {"first_name": "George","last_name" : "Baklarz"}
    ],
  "foreword": 
    {
      "primary": {"first_name": "Thomas","last_name" : "Hronis"}
    },
  "formats": ["Hardcover","Paperback","eBook","PDF"]
}
```

Each of the following path expressions results in either a value, an `ON EMPTY` clause being invoked, or an `ON ERROR` clause being invoked.

|#|Example | JSON Path | LAX | STRICT
|-:|:-------|:----------|:----|:------
|1|Find authors| `$.authors`| Value| Value
|2|Find any authors last name| `$.authors.last_name` | Value | ON ERROR
|3|Find any authors last name| `$.authors[*].last_name`| Value| Value
|4|Find any authors middle name| `$.authors.middle_name`| ON EMPTY| ON ERROR
|5|Find foreword first name| `$.foreword.primary.first_name`| Value| Value
|6|Find foreword (array) first name| `$.foreword.primary[0].first_name`| Value| ON ERROR
|7|Find formats element 1| `$.formats[1]`| Value| Value
|8|Find formats element 8| `$.formats[8]`| ON EMPTY| ON ERROR
|9|Find address field| `$.address`| ON EMPTY| ON ERROR
|10|Does anything in the first level have an array of first names| `$.*[*].first_name`| Value| ON ERROR

Most of the examples are straightforward, but here are a few examples that need further explanation.

*Example 2 & 3*

Examples numbers 2 & 3 both check for the existence of a `last_name` key in the authors object. The lax mode succeeds in example 2 while strict fails:
```sql
        JSON_EXISTS(info,'$.authors.last_name')
```

The path expression did not specify an array index for authors which causes the strict iteration of example 2 to fail while the `lax` iteration ignores this difference and the key is found. The `strict` format in example 3 succeeds because it explicitly includes the array specification.

*Example 5 & 6*

Examples number 5 & 6 check for the foreword authors first name. The first path works for both `lax` and `strict` mode while the second fails on strict mode.  
```sql
        JSON_EXISTS(info,'$.forward.primary[0].first_name')
```

The array specification after the primary field is ignored in `lax` mode while it causes `strict` mode to fail since it does not find an array.

*Example 10*

Example 10 requires some explanation on the use of the JSON path wildcard characters:
```sql
$.*[*].first_name
```

The patterns that are matched in the document are shown in the table below:

|Pattern|Path
|:------|:------
|`$.*`|`$.authors`
||**`$.foreword`**
||`$.formats`
|`$.*[*]`|`$.authors[]`
||**`$.formats[]`**
|`$.*[*].first_name`|`$.authors[*].first_name`

The `lax` specification returns a value while `strict` raises an error. The reason for the difference is that using `lax` will ignore structural differences in the path specification. The **foreword** object does not match `$.*[*]` and the **formats** object does not include a `first_name` field. Since there are two objects out of three that don't match, `strict` will fail, while `lax` will ignore the problem. 

Using the default `lax` specification for your JSON path expressions will generally result in the best interpretation of missing or incorrect values within a JSON document. The `lax` specification also ignores simple errors like incorrect array specifications in your path expressions. You should consider `strict` only if you want to ensure that there is no possibility of misinterpretation of the query or the data in the JSON document. Finally,
the `ON EMPTY` and `ON ERROR` clauses should be reviewed if you want your application to behave differently based on a structural difference within the document.

### JSON_EXISTS: Checking for Key-Value Pairs
JSON_EXISTS allows you to check whether or not a valid JSON key exists within a document for the provided search path. You can use the result of this function to determine if the contents of a JSON document are consistent with your expectations and to decide whether or not to take further action or retrieve the value. You can also use this function to validate that the JSON document is properly formed.

***JSON_EXISTS Syntax***
![JSON_EXISTS](images/JSON_EXISTS.png)

***Error Clause***
![JSON_EXISTS](images/je-error-clause.png)

The `JSON_EXISTS` function has three parmeters:
* The *json-expression* which is either a column name, a string expression, or a SQL variable
* The *json-path-expression* which describes which object we want to find
* An *error-clause* which determines what the function will do if it doesn't find the key

The *json-expression* and *json-path-expression* were discussed in a previous chapter on **JSON Path Expressions**.


The `ON ERROR` clause of the `JSON_EXISTS` function determines what value should be returned when an error occurs. The `ON ERROR` clause will only be invoked if you have an improper JSON document, or if you are using strict JSON path expressions and there are structural errors in the document (e.g. an invalid array index).

In the event there is an error in the document or there is a structural error, `JSON_EXISTS` will return a default value of `FALSE`. The function can return `FALSE`, `TRUE`, `UNKNOWN`, or `ERROR`. `UNKNOWN` is returned when the JSON expression is `NULL` and takes the form of a `NULL` value.

The `JSON_EXISTS` function will always return `FALSE` for missing keys or invalid documents whether you are using lax or strict mode within the path expression. The reason for this behavior is that the default `ON ERROR` clause returns `FALSE` rather than raising an error.

### Load Db2 Extensions and Connect to the Database
The `connection` notebook contains the `CONNECT` statement which allows access to the `SAMPLE` database. If you need to modify the connection information, edit the `connection.ipynb` notebook.

In [ ]:
%run db2.ipynb
%run connection.ipynb

***Example: Invalid Document (lax or strict)***

The following example will return a value of `FALSE` (default path assumption is lax).

In [ ]:
%sql VALUES JSON_EXISTS('{"name" : George}','$.name');

The error was raised because the document was invalid JSON. The value George was assumed to be a numeric value because it was not enclosed in double quotes. This would also happen if strict had been used.

***Raise an ERROR Condition on Invalid Document (lax or strict)***

Changing the `ON ERROR` condition to `ERROR` will cause the function to generate an error condition.

In [ ]:
%sql VALUES JSON_EXISTS('{"name": George}','$.name' ERROR ON ERROR);

***Example: Empty Document or Path Expression***

Using an empty document will result in a `NULL` value being returned by the statement. 

In [ ]:
%sql VALUES JSON_EXISTS(null,'$.name')

**Note:** The Python display equivalent to **null** is **None**.

Using a `NULL` or an empty path expression will result in `FALSE` being returned by the function.

In [ ]:
%sql VALUES JSON_EXISTS('{"name": "George"}', null);

**Note:** Depending on the Db2 client settings, the value `False` is returned as either a zero or **false**.

***Example: Check Whether any Authors Exist in the Document***

The following examples will use the books document.

In [ ]:
books = {
   "authors": [{"first_name": "Paul",  "last_name" : "Bird"},
               {"first_name": "George","last_name" : "Baklarz"}],
   "foreword": {
       "primary": {"first_name": "Thomas","last_name" : "Hronis"}
   },
   "formats": ["Hardcover","Paperback","eBook","PDF"]
}

This statement will check to see if any authors exists in the document. You can use either lax or strict in this example and they will both return `TRUE` since the document contains the authors key.

In [ ]:
%sql VALUES JSON_EXISTS(:books,'strict $.authors');

***Check If an Author Has a Last Name***

This example checks to see if there is a last name associated with an author. The issue with this request is that the authors object is an array of values. Using lax will result in a result of TRUE since the structural differences are ignored.

In [ ]:
%sql VALUES JSON_EXISTS(:books,'lax $.authors.last_name');

Attempting to run this with strict will result in `FALSE` being returned because the path expression is missing the array aspect of the document.

In [ ]:
%sql VALUES JSON_EXISTS(:books,'strict $.authors.last_name');

If we correct the path expression to now include the array specifier, it will work. In this case, using the modifier will work since the path exists in the document, while in the previous example the path does not exist unless you ignore the structural issues.

In [ ]:
%sql VALUES JSON_EXISTS(:books,'strict $.authors[*].last_name');

***Find the First Name of the Author Who Wrote the Foreword*** 

This is another example that demonstrates how structural differences are handled by lax versus strict. There is only one author that wrote the foreword, but the JSON path expression includes an array specifier. Using lax results in a `TRUE` result since the array specification is ignored.

In [ ]:
%sql VALUES JSON_EXISTS(:books,'lax $.foreword.primary[0].first_name');

Attempting the same function with strict mode will result in `FALSE` being returned because the path specifies an array that does not exist in the document.

In [ ]:
%sql VALUES JSON_EXISTS(:books,'strict $.foreword.primary[0].first_name');

***Checking for a Book Type that is Outside the Bounds***

If you check for an array value that is outside of the bounds of the object, a `FALSE` value will be returned when using either lax or strict mode.

In [ ]:
%sql VALUES JSON_EXISTS(:books,'strict $.formats[8]');

***Complex Request to Check for Any First Names***

The following SQL attempts to determine if there is any key (`$.*`) that contains an array (`[*]`), and that array includes a first_name field. 

In [ ]:
%sql VALUES JSON_EXISTS(:books,'lax $.*[*].first_name')

If the path expression was changed to strict, the return value would be `FALSE`. The following table displays the portions of the books document that are matched by the path expression.

|Pattern|Path
|:------|:-----
|`$.*`|`$.authors`
||**`$.foreword`**
||`$.formats`
|`$.*[*`]|`$.authors[]`
||**`$.formats`**
|`$.*[*].first_name`|`$.authors[*].first_name`

The reason for the difference is that using lax will return `TRUE` if it finds at least one match in the document while strict requires that all items found must match. The foreword object is not an array, and the formats object does not include a first_name field. Since there are multiple differences, strict will fail, while lax will ignore the problems.

Using the default lax specification for your `JSON_EXISTS` path expressions will generally result in the best interpretation of missing or incorrect values within a JSON document. The lax specification also ignores simple errors like incorrect array specifications in your path expressions. You should consider strict only if you want to ensure that there is no possibility of misinterpretation of the query or the data in the JSON document. 

### Summary
The `JSON_EXISTS` function can be used to determine whether key fields are present within a document. You can use the result of this function to determine if the contents of a JSON document are consistent with your expectations and to decide whether or not to take further action or retrieve the value.

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]